In [ ]:
# 프롬프트 조정 
# 출력 답변에 채용 공고 개수와 비율 계산하는 수식 출력 x


In [ ]:
import openai
import sqlite3
import ast  # 문자열로 저장된 리스트 변환용

# ✅ OpenAI API 키 설정
openai.api_key = ""

# ✅ SQLite 데이터베이스 연결
db_path = "c:/Users/asia/Desktop/asia_camp_2024/final_project_asia/3.ETL_DB_Load(Career,Language)/asia (2-2).db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ✅ SQL 쿼리 실행 
sql_query = """
WITH career_data AS (
    SELECT 'PM' AS category, career FROM processing WHERE duty LIKE '%PM%'
    UNION ALL
    SELECT '데이터 직무' AS category, career FROM processing WHERE duty LIKE '%데이터 직무%'
    UNION ALL
    SELECT '백엔드' AS category, career FROM processing WHERE duty LIKE '%백엔드%'
    UNION ALL
    SELECT '인프라 엔지니어' AS category, career FROM processing WHERE duty LIKE '%인프라 엔지니어%'
    UNION ALL
    SELECT '앱 개발자' AS category, career FROM processing WHERE duty LIKE '%앱 개발자%'
    UNION ALL
    SELECT '게임' AS category, career FROM processing WHERE duty LIKE '%게임%'
    UNION ALL
    SELECT 'AI' AS category, career FROM processing WHERE duty LIKE '%AI%'
    UNION ALL
    SELECT '임베디드' AS category, career FROM processing WHERE duty LIKE '%임베디드%'
    UNION ALL
    SELECT '프론트 엔드' AS category, career FROM processing WHERE duty LIKE '%프론트 엔드%'
    UNION ALL
    SELECT 'QA' AS category, career FROM processing WHERE duty LIKE '%QA%'
    UNION ALL
    SELECT '데이터 분석' AS category, career FROM processing WHERE duty LIKE '%데이터 분석%'
    UNION ALL
    SELECT 'VR' AS category, career FROM processing WHERE duty LIKE '%VR%'
    UNION ALL
    SELECT '시스템' AS category, career FROM processing WHERE duty LIKE '%시스템%'
    UNION ALL
    SELECT '블록체인' AS category, career FROM processing WHERE duty LIKE '%블록체인%'
    UNION ALL
    SELECT 'ERP' AS category, career FROM processing WHERE duty LIKE '%ERP%'
    UNION ALL
    SELECT '언어별 개발자' AS category, career FROM processing WHERE duty LIKE '%언어별 개발자%'
),
category_total AS (
    SELECT category, COUNT(*) AS total FROM career_data GROUP BY category
)

SELECT
    c.category AS 직무,
    c.career AS 연차,
    COUNT(*) AS 채용_공고_개수,
    ROUND(COUNT(*) * 100.0 / ct.total, 2) AS 직무_내_비율
FROM career_data c
JOIN category_total ct ON c.category = ct.category
GROUP BY c.category, c.career, ct.total
ORDER BY c.category, c.career;
"""

cursor.execute(sql_query)
career_data = cursor.fetchall()
print(career_data)

# ✅ 직무별 career 데이터 저장
job_career_dict = {}
for duty, career, job_count, ratio in career_data:  # ✅ 4개 컬럼 올바르게 언패킹
    if duty not in job_career_dict:
        job_career_dict[duty] = []
    job_career_dict[duty].append((career, ratio))  # ✅ 연차, 확률값(%)만 저장

# ✅ 모든 직무에 대해 GPT-4 분석 수행
for duty, career_info in job_career_dict.items():
    # ✅ 기존 career 컬럼 값 조회
    cursor.execute("SELECT career FROM duty_analysis WHERE duty = ?;", (duty,))
    existing_career_data = cursor.fetchone()

    # ✅ 연차별 데이터 변환 (채용 건수 제거, 확률만 제공)
    if career_info:
        job_details = "\n".join([
            f"- {'신입사원' if c[0] == 0 else f'{c[0]}년차'}: {c[1]}%"
            for c in career_info
        ])
    else:
        job_details = f"{duty} 직무의 채용 데이터가 없습니다."

    print(f"🎯 {duty} 직무 데이터 확인:\n{job_details}")

    # ✅ GPT-4 프롬프트 생성
    prompt = f"""
    "{duty}" 직무의 채용 공고 데이터를 분석해주세요.

    [직무별 연차별 채용 공고 비율]
    {job_details}

    위 데이터를 기반으로 다음을 분석해주세요:
    1. 가장 높은 비율을 차지하는 경력 연차와 해당 비율
    2. 두 번째로 높은 비율을 차지하는 경력 연차와 해당 비율
    3. 가장 낮은 비율을 차지하는 경력 연차와 해당 비율
    4. {duty} 직무에서 경력별 채용 비율을 기반으로 한 특징 및 트렌드 분석

    ❗ 단, "경력 0년"은 "신입사원"으로 변환해서 출력해주세요.
    ❗ 모든 문장은 "입니다"로 끝나도록 통일해주세요.
    ❗ 설명은 간결하게 요약해서 제공해주세요.
    ❗ "신입사원"이라는 단어가 변형되지 않도록 정확하게 출력해주세요.
    """

    try:
        # ✅ GPT-4 API 호출
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}]
        )
        gpt4_summary = response.choices[0].message.content.strip()

        print(f"✅ {duty} 직무 최종 응답:\n{gpt4_summary}")

        # ✅ 기존 career 컬럼 값 가져오기 (이미지 경로 유지)
        if existing_career_data and existing_career_data[0]:
            try:
                career_list = ast.literal_eval(existing_career_data[0])  # 문자열 → 리스트 변환
                if isinstance(career_list, list) and career_list:
                    image_path = career_list[0]  # 기존 이미지 경로 유지
                    career_list = [image_path, gpt4_summary]  # GPT 설명 업데이트
                else:
                    career_list = [gpt4_summary]  # 기존 데이터가 리스트가 아니면 새로 생성
            except:
                career_list = [gpt4_summary]  # 변환 실패 시 GPT 설명만 추가
        else:
            career_list = [gpt4_summary]  # 기존 값이 없으면 새 리스트 생성

        # ✅ 업데이트할 값 변환 (리스트 → 문자열)
        updated_text = str(career_list)

        cursor.execute("UPDATE duty_analysis SET career = ? WHERE duty = ?;", (updated_text, duty))
        conn.commit()

    except Exception as e:
        print(f"❌ {duty} 직무 요약 실패: {e}")

# ✅ DB 연결 종료
conn.close()


[('AI', 0, 90, 20.04), ('AI', 1, 45, 10.02), ('AI', 2, 75, 16.7), ('AI', 3, 132, 29.4), ('AI', 4, 12, 2.67), ('AI', 5, 63, 14.03), ('AI', 6, 8, 1.78), ('AI', 7, 13, 2.9), ('AI', 8, 4, 0.89), ('AI', 10, 7, 1.56), ('ERP', 0, 1, 2.7), ('ERP', 1, 3, 8.11), ('ERP', 2, 4, 10.81), ('ERP', 3, 11, 29.73), ('ERP', 4, 8, 21.62), ('ERP', 5, 6, 16.22), ('ERP', 7, 4, 10.81), ('PM', 0, 12, 7.89), ('PM', 1, 12, 7.89), ('PM', 2, 11, 7.24), ('PM', 3, 36, 23.68), ('PM', 4, 3, 1.97), ('PM', 5, 48, 31.58), ('PM', 6, 5, 3.29), ('PM', 7, 11, 7.24), ('PM', 8, 7, 4.61), ('PM', 10, 7, 4.61), ('QA', 0, 26, 16.05), ('QA', 1, 19, 11.73), ('QA', 2, 20, 12.35), ('QA', 3, 49, 30.25), ('QA', 4, 10, 6.17), ('QA', 5, 25, 15.43), ('QA', 6, 3, 1.85), ('QA', 7, 7, 4.32), ('QA', 8, 1, 0.62), ('QA', 10, 2, 1.23), ('VR', 0, 3, 20.0), ('VR', 2, 3, 20.0), ('VR', 3, 5, 33.33), ('VR', 4, 1, 6.67), ('VR', 5, 3, 20.0), ('게임', 0, 3, 13.04), ('게임', 2, 3, 13.04), ('게임', 3, 10, 43.48), ('게임', 5, 7, 30.43), ('데이터 분석', 0, 27, 20.45), ('데

In [ ]:
[('AI', 0, 90, 20.04), ('AI', 1, 45, 10.02), ('AI', 2, 75, 16.7), ('AI', 3, 132, 29.4), ('AI', 4, 12, 2.67), ('AI', 5, 63, 14.03), ('AI', 6, 8, 1.78), ('AI', 7, 13, 2.9), ('AI', 8, 4, 0.89), ('AI', 10, 7, 1.56), ('ERP', 0, 1, 2.7), ('ERP', 1, 3, 8.11), ('ERP', 2, 4, 10.81), ('ERP', 3, 11, 29.73), ('ERP', 4, 8, 21.62), ('ERP', 5, 6, 16.22), ('ERP', 7, 4, 10.81), ('PM', 0, 12, 7.89), ('PM', 1, 12, 7.89), ('PM', 2, 11, 7.24), ('PM', 3, 36, 23.68), ('PM', 4, 3, 1.97), ('PM', 5, 48, 31.58), ('PM', 6, 5, 3.29), ('PM', 7, 11, 7.24), ('PM', 8, 7, 4.61), ('PM', 10, 7, 4.61), ('QA', 0, 26, 16.05), ('QA', 1, 19, 11.73), ('QA', 2, 20, 12.35), ('QA', 3, 49, 30.25), ('QA', 4, 10, 6.17), ('QA', 5, 25, 15.43), ('QA', 6, 3, 1.85), ('QA', 7, 7, 4.32), ('QA', 8, 1, 0.62), ('QA', 10, 2, 1.23), ('VR', 0, 3, 20.0), ('VR', 2, 3, 20.0), ('VR', 3, 5, 33.33), ('VR', 4, 1, 6.67), ('VR', 5, 3, 20.0), ('게임', 0, 3, 13.04), ('게임', 2, 3, 13.04), ('게임', 3, 10, 43.48), ('게임', 5, 7, 30.43), ('데이터 분석', 0, 27, 20.45), ('데이터 분석', 1, 14, 10.61), ('데이터 분석', 2, 22, 16.67), ('데이터 분석', 3, 38, 28.79), ('데이터 분석', 4, 2, 1.52), ('데이터 분석', 5, 17, 12.88), ('데이터 분석', 6, 1, 0.76), ('데이터 분석', 7, 3, 2.27), ('데이터 분석', 8, 1, 0.76), ('데이터 분석', 10, 7, 5.3), ('데이터 직무', 0, 53, 14.36), ('데이터 직무', 1, 22, 5.96), ('데이터 직무', 2, 49, 13.28), ('데이터 직무', 3, 101, 27.37), ('데이터 직무', 4, 21, 5.69), ('데이터 직무', 5, 74, 20.05), ('데이터 직무', 6, 6, 1.63), ('데이터 직무', 7, 23, 6.23), ('데이터 직무', 8, 8, 2.17), ('데이터 직무', 10, 9, 2.44), ('데이터 직무', 11, 2, 0.54), ('데이터 직무', 15, 1, 0.27), ('백엔드', 0, 110, 8.4), ('백엔드', 1, 73, 5.57), ('백엔드', 2, 128, 9.77), ('백엔드', 3, 429, 32.75), ('백엔드', 4, 77, 5.88), ('백엔드', 5, 296, 22.6), ('백엔드', 6, 39, 2.98), ('백엔드', 7, 83, 6.34), ('백엔드', 8, 30, 2.29), ('백엔드', 9, 3, 0.23), ('백엔드', 10, 38, 2.9), ('백엔드', 12, 1, 0.08), ('백엔드', 15, 3, 0.23), ('블록체인', 0, 9, 13.04), ('블록체인', 1, 10, 14.49), ('블록체인', 2, 9, 13.04), ('블록체인', 3, 16, 23.19), ('블록체인', 4, 1, 1.45), ('블록체인', 5, 15, 21.74), ('블록체인', 7, 2, 2.9), ('블록체인', 8, 7, 10.14), ('시스템', 0, 34, 12.45), ('시스템', 1, 22, 8.06), ('시스템', 2, 18, 6.59), ('시스템', 3, 104, 38.1), ('시스템', 4, 15, 5.49), ('시스템', 5, 51, 18.68), ('시스템', 6, 4, 1.47), ('시스템', 7, 10, 3.66), ('시스템', 8, 4, 1.47), ('시스템', 9, 1, 0.37), ('시스템', 10, 8, 2.93), ('시스템', 13, 1, 0.37), ('시스템', 15, 1, 0.37), ('앱 개발자', 0, 32, 11.15), ('앱 개발자', 1, 28, 9.76), ('앱 개발자', 2, 34, 11.85), ('앱 개발자', 3, 99, 34.49), ('앱 개발자', 4, 14, 4.88), ('앱 개발자', 5, 58, 20.21), ('앱 개발자', 6, 5, 1.74), ('앱 개발자', 7, 14, 4.88), ('앱 개발자', 8, 1, 0.35), ('앱 개발자', 10, 2, 0.7), ('언어별 개발자', 0, 127, 11.05), ('언어별 개발자', 1, 91, 7.92), ('언어별 개발자', 2, 132, 11.49), ('언어별 개발자', 3, 344, 29.94), ('언어별 개발자', 4, 65, 5.66), ('언어별 개발자', 5, 249, 21.67), ('언어별 개발자', 6, 21, 1.83), ('언어별 개발자', 7, 60, 5.22), ('언어별 개발자', 8, 26, 2.26), ('언어별 개발자', 9, 3, 0.26), ('언어별 개발자', 10, 26, 2.26), ('언어별 개발자', 15, 5, 0.44), ('인프라 엔지니어', 0, 129, 10.42), ('인프라 엔지니어', 1, 102, 8.24), ('인프라 엔지니어', 2, 122, 9.85), ('인프라 엔지니어', 3, 394, 31.83), ('인프라 엔지니어', 4, 64, 5.17), ('인프라 엔지니어', 5, 264, 21.32), ('인프라 엔지니어', 6, 29, 2.34), ('인프라 엔지니어', 7, 56, 4.52), ('인프라 엔지니어', 8, 29, 2.34), ('인프라 엔지니어', 9, 3, 0.24), ('인프라 엔지니어', 10, 40, 3.23), ('인프라 엔지니어', 12, 1, 0.08), ('인프라 엔지니어', 13, 1, 0.08), ('인프라 엔지니어', 15, 4, 0.32), ('임베디드', 0, 63, 19.57), ('임베디드', 1, 27, 8.39), ('임베디드', 2, 50, 15.53), ('임베디드', 3, 86, 26.71), ('임베디드', 4, 17, 5.28), ('임베디드', 5, 48, 14.91), ('임베디드', 6, 2, 0.62), ('임베디드', 7, 6, 1.86), ('임베디드', 8, 7, 2.17), ('임베디드', 9, 1, 0.31), ('임베디드', 10, 13, 4.04), ('임베디드', 15, 2, 0.62), ('프론트 엔드', 0, 84, 9.0), ('프론트 엔드', 1, 67, 7.18), ('프론트 엔드', 2, 103, 11.04), ('프론트 엔드', 3, 300, 32.15), ('프론트 엔드', 4, 55, 5.89), ('프론트 엔드', 5, 194, 20.79), ('프론트 엔드', 6, 25, 2.68), ('프론트 엔드', 7, 55, 5.89), ('프론트 엔드', 8, 15, 1.61), ('프론트 엔드', 9, 4, 0.43), ('프론트 엔드', 10, 30, 3.22), ('프론트 엔드', 15, 1, 0.11)]
🎯 AI 직무 데이터 확인:
- 신입사원: 20.04%
- 1년차: 10.02%
- 2년차: 16.7%
- 3년차: 29.4%
- 4년차: 2.67%
- 5년차: 14.03%
- 6년차: 1.78%
- 7년차: 2.9%
- 8년차: 0.89%
- 10년차: 1.56%
✅ AI 직무 최종 응답:
1. 가장 높은 비율을 차지하는 경력 연차는 3년차이며, 해당 비율은 29.4% 입니다.
2. 두 번째로 높은 비율을 차지하는 경력 연차는 5년차이며, 해당 비율은 14.03% 입니다.
3. 가장 낮은 비율을 차지하는 경력 연차는 8년차이며, 해당 비율은 0.89% 입니다.
4. AI 직무에서는 3년차와 5년차가 많이 채용되고 있으며, 신입사원과 2년차의 비율도 높은 편이지만, 8년차와 10년차의 채용 비율은 매우 낮은 편입니다. 이를 통해 AI 기술이 빠르게 변화하고 있으며, 중간 경력층이나 고급 경력층보다는 경력이 어느 정도 쌓인 신입사원이나 중간 경력층이 더 많이 선호되는 경향이 있다는 것을 알 수 있습니다.
🎯 ERP 직무 데이터 확인:
- 신입사원: 2.7%
- 1년차: 8.11%
- 2년차: 10.81%
- 3년차: 29.73%
- 4년차: 21.62%
- 5년차: 16.22%
- 7년차: 10.81%
✅ ERP 직무 최종 응답:
1. 가장 높은 비율을 차지하는 경력 연차는 3년차이며, 해당 비율은 29.73%입니다.
2. 두 번째로 높은 비율을 차지하는 경력 연차는 4년차이며, 해당 비율은 21.62%입니다.
3. 가장 낮은 비율을 차지하는 경력 연차는 7년차이며, 해당 비율은 10.81%입니다.
4. ERP 직무에서는 3년차의 채용 비중이 가장 높고, 그 다음으로 4년차가 높은 편이며, 상대적으로 신입사원과 7년차는 채용 비율이 낮은 편입니다. 경력이 쌓일수록 채용 비율이 높아지는 경향이 있습니다.
🎯 PM 직무 데이터 확인:
- 신입사원: 7.89%
- 1년차: 7.89%
- 2년차: 7.24%
- 3년차: 23.68%
- 4년차: 1.97%
- 5년차: 31.58%
- 6년차: 3.29%
- 7년차: 7.24%
- 8년차: 4.61%
- 10년차: 4.61%
✅ PM 직무 최종 응답:
1. 가장 높은 비율을 차지하는 경력 연차는 5년차이며, 해당 비율은 31.58%입니다.
2. 두 번째로 높은 비율을 차지하는 경력 연차는 3년차이며, 해당 비율은 23.68%입니다.
3. 가장 낮은 비율을 차지하는 경력 연차는 4년차이며, 해당 비율은 1.97%입니다.
4. PM 직무에서는 경력이 쌓일수록 채용 비율이 높아지다가 5년차에서 가장 높은 비율을 기록하고 있으며, 신입사원과 1년차 역시 높은 비율을 보여줍니다. 그러나 4년차에서는 낮은 비율을 보이고 있습니다. 이를 통해 PM 직무에서는 상대적으로 중간 단계의 경력자보다는 신입사원과 경력이 쌓인 경력자를 선호하는 경향이 있습니다.
🎯 QA 직무 데이터 확인:
- 신입사원: 16.05%
- 1년차: 11.73%
- 2년차: 12.35%
- 3년차: 30.25%
- 4년차: 6.17%
- 5년차: 15.43%
- 6년차: 1.85%
- 7년차: 4.32%
- 8년차: 0.62%
- 10년차: 1.23%
✅ QA 직무 최종 응답:
1. 가장 높은 비율을 차지하는 경력 연차는 3년차이며, 해당 비율은 30.25% 입니다.

2. 두 번째로 높은 비율을 차지하는 경력 연차는 5년차이며, 해당 비율은 15.43% 입니다.

3. 가장 낮은 비율을 차지하는 경력 연차는 8년차이며, 해당 비율은 0.62% 입니다.

4. QA 직무에서는 신입사원과 3년차 이상의 경력자를 주로 채용하는 트렌드가 있습니다. 특히 3년차가 가장 높은 비율을 차지하고 있으며, 경력이 쌓일수록 채용 비율이 감소하는 양상을 보이고 있습니다.
🎯 VR 직무 데이터 확인:
- 신입사원: 20.0%
- 2년차: 20.0%
- 3년차: 33.33%
- 4년차: 6.67%
- 5년차: 20.0%
✅ VR 직무 최종 응답:
1. 가장 높은 비율을 차지하는 경력 연차는 3년차이며, 해당 비율은 33.33%입니다.
2. 두 번째로 높은 비율을 차지하는 경력 연차는 신입사원(경력 0년)이며, 해당 비율은 20.0%입니다.
3. 가장 낮은 비율을 차지하는 경력 연차는 4년차이며, 해당 비율은 6.67%입니다.
4. VR 직무에서는 경력 3년차의 채용 비율이 가장 높고, 그 다음으로는 신입사원의 채용 비율이 높은 특징이 있습니다. 경력 4년차의 채용 비율은 상대적으로 낮은 편입니다.
🎯 게임 직무 데이터 확인:
- 신입사원: 13.04%
- 2년차: 13.04%
- 3년차: 43.48%
- 5년차: 30.43%
✅ 게임 직무 최종 응답:
1. 가장 높은 비율을 차지하는 경력 연차는 3년차로, 해당 비율은 43.48%입니다.
2. 두 번째로 높은 비율을 차지하는 경력 연차는 5년차로, 해당 비율은 30.43%입니다.
3. 가장 낮은 비율을 차지하는 경력 연차는 신입사원(0년차)으로, 해당 비율은 13.04%입니다.
4. 게임 직무에서는 경력자를 우대하는 경향이 있으며, 특히 3년차의 채용 비율이 가장 높은 것으로 보아 중간 경력자를 중심으로 채용하는 경향이 있습니다. 신입사원과 2년차의 비율은 비교적 낮은 편이며, 5년차의 비율도 상당한 비중을 차지하고 있습니다.
🎯 데이터 분석 직무 데이터 확인:
- 신입사원: 20.45%
- 1년차: 10.61%
- 2년차: 16.67%
- 3년차: 28.79%
- 4년차: 1.52%
- 5년차: 12.88%
- 6년차: 0.76%
- 7년차: 2.27%
- 8년차: 0.76%
- 10년차: 5.3%
✅ 데이터 분석 직무 최종 응답:
1. 가장 높은 비율을 차지하는 경력 연차는 3년차이며 해당 비율은 28.79%입니다.

2. 두 번째로 높은 비율을 차지하는 경력 연차는 5년차이며 해당 비율은 12.88%입니다.

3. 가장 낮은 비율을 차지하는 경력 연차는 6년차와 8년차로 동일하며 해당 비율은 각각 0.76%입니다.

4. 데이터 분석 직무에서는 주로 3년차가 가장 많이 채용되며, 이어서 5년차가 많이 채용되는 특징이 있습니다. 반면에 6년차와 8년차는 매우 낮은 비율로 채용되는 경향이 있습니다. 데이터 분석 분야는 경력이 쌓일수록 채용이 쉽지 않은 트렌드를 보이고 있습니다.
🎯 데이터 직무 직무 데이터 확인:
- 신입사원: 14.36%
- 1년차: 5.96%
- 2년차: 13.28%
- 3년차: 27.37%
- 4년차: 5.69%
- 5년차: 20.05%
- 6년차: 1.63%
- 7년차: 6.23%
- 8년차: 2.17%
- 10년차: 2.44%
- 11년차: 0.54%
- 15년차: 0.27%
✅ 데이터 직무 직무 최종 응답:
1. 가장 높은 비율을 차지하는 경력 연차는 3년차로, 해당 비율은 27.37% 입니다.
2. 두 번째로 높은 비율을 차지하는 경력 연차는 5년차로, 해당 비율은 20.05% 입니다.
3. 가장 낮은 비율을 차지하는 경력 연차는 15년차로, 해당 비율은 0.27% 입니다.
4. 데이터 직무에서는 경력이 쌓일수록 채용 비율이 감소하는 트렌드를 보이며, 신입사원과 3년차의 비율이 상대적으로 높은 것이 특징입니다.
🎯 백엔드 직무 데이터 확인:
- 신입사원: 8.4%
- 1년차: 5.57%
- 2년차: 9.77%
- 3년차: 32.75%
- 4년차: 5.88%
- 5년차: 22.6%
- 6년차: 2.98%
- 7년차: 6.34%
- 8년차: 2.29%
- 9년차: 0.23%
- 10년차: 2.9%
- 12년차: 0.08%
- 15년차: 0.23%
✅ 백엔드 직무 최종 응답:
1. 가장 높은 비율을 차지하는 경력 연차는 3년차로, 해당 비율은 32.75%입니다.
2. 두 번째로 높은 비율을 차지하는 경력 연차는 5년차로, 해당 비율은 22.6%입니다.
3. 가장 낮은 비율을 차지하는 경력 연차는 12년차와 15년차로, 각각의 비율은 0.08%와 0.23%입니다.
4. 백엔드 직무에서는 3년차와 5년차가 높은 비율을 차지하고 있으며, 경력이 쌓일수록 채용 비율이 감소하는 추세를 보입니다. "신입사원"과 3년차, 5년차에 초점을 맞춰 채용을 진행하는 경향이 있습니다.
🎯 블록체인 직무 데이터 확인:
- 신입사원: 13.04%
- 1년차: 14.49%
- 2년차: 13.04%
- 3년차: 23.19%
- 4년차: 1.45%
- 5년차: 21.74%
- 7년차: 2.9%
- 8년차: 10.14%
✅ 블록체인 직무 최종 응답:
1. 가장 높은 비율을 차지하는 경력 연차는 3년차로, 해당 비율은 23.19%입니다.
2. 두 번째로 높은 비율을 차지하는 경력 연차는 5년차로, 해당 비율은 21.74%입니다.
3. 가장 낮은 비율을 차지하는 경력 연차는 4년차로, 해당 비율은 1.45%입니다.
4. 블록체인 직무에서는 경력 3년차와 5년차의 채용 비율이 높은 편이며, 신입사원과 4년차의 채용 비율은 낮은 편입니다. 이는 블록체인 분야에서는 경력이 있는 전문가들을 선호하는 경향이 있다고 볼 수 있습니다.
🎯 시스템 직무 데이터 확인:
- 신입사원: 12.45%
- 1년차: 8.06%
- 2년차: 6.59%
- 3년차: 38.1%
- 4년차: 5.49%
- 5년차: 18.68%
- 6년차: 1.47%
- 7년차: 3.66%
- 8년차: 1.47%
- 9년차: 0.37%
- 10년차: 2.93%
- 13년차: 0.37%
- 15년차: 0.37%
✅ 시스템 직무 최종 응답:
1. 가장 높은 비율을 차지하는 경력 연차는 3년차이며, 해당 비율은 38.1%입니다.

2. 두 번째로 높은 비율을 차지하는 경력 연차는 5년차이며, 해당 비율은 18.68%입니다.

3. 가장 낮은 비율을 차지하는 경력 연차는 9년차, 13년차, 15년차이며, 해당 비율은 0.37%입니다.

4. 시스템 직무에서는 신입사원 및 3년차의 채용 비율이 높은 편이며, 상대적으로 9년차 이상의 경력자들의 채용 비율은 낮은 편입니다. 따라서, 신입사원과 경력 3년차의 채용에 더욱 초점을 맞춘 채용 정책을 가지고 있습니다.
🎯 앱 개발자 직무 데이터 확인:
- 신입사원: 11.15%
- 1년차: 9.76%
- 2년차: 11.85%
- 3년차: 34.49%
- 4년차: 4.88%
- 5년차: 20.21%
- 6년차: 1.74%
- 7년차: 4.88%
- 8년차: 0.35%
- 10년차: 0.7%
✅ 앱 개발자 직무 최종 응답:
1. 가장 높은 비율을 차지하는 경력 연차는 3년차이며, 해당 비율은 34.49%입니다.
2. 두 번째로 높은 비율을 차지하는 경력 연차는 5년차이며, 해당 비율은 20.21%입니다.
3. 가장 낮은 비율을 차지하는 경력 연차는 8년차이며, 해당 비율은 0.35%입니다.
4. 앱 개발자 직무에서는 3년차와 5년차의 경력자가 많이 채용되는 경향이 있으며, 신입사원 및 1년차는 비교적 적게 채용되는 편입니다. 또한, 8년차 이상의 고급 경력자는 매우 드물게 채용되는 것으로 나타납니다.
🎯 언어별 개발자 직무 데이터 확인:
- 신입사원: 11.05%
- 1년차: 7.92%
- 2년차: 11.49%
- 3년차: 29.94%
- 4년차: 5.66%
- 5년차: 21.67%
- 6년차: 1.83%
- 7년차: 5.22%
- 8년차: 2.26%
- 9년차: 0.26%
- 10년차: 2.26%
- 15년차: 0.44%
✅ 언어별 개발자 직무 최종 응답:
1. 가장 높은 비율을 차지하는 경력 연차는 3년차로, 해당 비율은 29.94% 입니다.
2. 두 번째로 높은 비율을 차지하는 경력 연차는 5년차로, 해당 비율은 21.67% 입니다.
3. 가장 낮은 비율을 차지하는 경력 연차는 9년차와 15년차로, 해당 비율은 각각 0.26%와 0.44% 입니다.
4. 언어별 개발자 직무에서는 주로 3년차와 5년차의 경력자를 채용하는 경향이 있으며, 신입사원 채용 비율도 상대적으로 높은 편입니다. 반면에 9년차와 15년차의 경력자를 채용하는 경우는 매우 드물다는 특징이 있습니다.
🎯 인프라 엔지니어 직무 데이터 확인:
- 신입사원: 10.42%
- 1년차: 8.24%
- 2년차: 9.85%
- 3년차: 31.83%
- 4년차: 5.17%
- 5년차: 21.32%
- 6년차: 2.34%
- 7년차: 4.52%
- 8년차: 2.34%
- 9년차: 0.24%
- 10년차: 3.23%
- 12년차: 0.08%
- 13년차: 0.08%
- 15년차: 0.32%
✅ 인프라 엔지니어 직무 최종 응답:
1. 가장 높은 비율을 차지하는 경력 연차는 3년차이며, 해당 비율은 31.83%입니다.
2. 두 번째로 높은 비율을 차지하는 경력 연차는 5년차이며, 해당 비율은 21.32%입니다.
3. 가장 낮은 비율을 차지하는 경력 연차는 13년차와 12년차로 동일하며, 해당 비율은 0.08%입니다.
4. 인프라 엔지니어 직무에서는 3년차와 5년차가 가장 많이 채용되고 있으며, 경력이 쌓일수록 채용 비율이 감소하는 경향이 있습니다. 신입사원과 1년차도 일정 비율을 차지하지만, 그 이후로는 경력이 있는 지원자를 선호하는 경향이 있습니다.
🎯 임베디드 직무 데이터 확인:
- 신입사원: 19.57%
- 1년차: 8.39%
- 2년차: 15.53%
- 3년차: 26.71%
- 4년차: 5.28%
- 5년차: 14.91%
- 6년차: 0.62%
- 7년차: 1.86%
- 8년차: 2.17%
- 9년차: 0.31%
- 10년차: 4.04%
- 15년차: 0.62%
✅ 임베디드 직무 최종 응답:
1. 가장 높은 비율을 차지하는 경력 연차는 3년차이며, 해당 비율은 26.71% 입니다.

2. 두 번째로 높은 비율을 차지하는 경력 연차는 5년차이며, 해당 비율은 14.91% 입니다.

3. 가장 낮은 비율을 차지하는 경력 연차는 9년차이며, 해당 비율은 0.31% 입니다.

4. 임베디드 직무에서는 주로 3년차의 경력자를 채용하는 경향이 있으며, 그 다음으로는 5년차의 경력자를 채용하는 것을 볼 수 있습니다. 또한, 9년차 이상의 고급 경력자들은 상대적으로 적게 채용되고 있습니다.
🎯 프론트 엔드 직무 데이터 확인:
- 신입사원: 9.0%
- 1년차: 7.18%
- 2년차: 11.04%
- 3년차: 32.15%
- 4년차: 5.89%
- 5년차: 20.79%
- 6년차: 2.68%
- 7년차: 5.89%
- 8년차: 1.61%
- 9년차: 0.43%
- 10년차: 3.22%
- 15년차: 0.11%
✅ 프론트 엔드 직무 최종 응답:
1. 가장 높은 비율을 차지하는 경력 연차는 3년차이며, 해당 비율은 32.15%입니다.
2. 두 번째로 높은 비율을 차지하는 경력 연차는 5년차이며, 해당 비율은 20.79%입니다.
3. 가장 낮은 비율을 차지하는 경력 연차는 15년차(경력 15년)이며, 해당 비율은 0.11%입니다.
4. 프론트 엔드 직무에서는 경력이 3년차인 개발자를 가장 선호하는 경향이 있으며, 그 다음으로는 5년차 경력의 개발자를 선호하는 것으로 보입니다. 신입사원과 1~2년차 경력 개발자의 채용 비율은 비교적 낮은 편이고, 10년차 이상 경력 개발자의 채용 비율도 낮은 편에 속합니다.